# Tutorial for scoring benchmarking

This code loads true and modeled data, then compares the times and results of various scoring approaches. This is a useful companion to regression benchmarking, because the accuracy and duration of cross-validated ridge constraint estimation methods depend on the scoring approach.

Written by Michael Sokoletsky, 2023.


### Imports and opts

In [32]:
import time

import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.metrics import make_scorer
from scipy.optimize import fminbound
from tqdm.notebook import tqdm

import regbench as rb

opts = {}
# Recording parameters
opts['dir'] = 'D:\Churchland\Widefield' # Path to directory in which the recording file is located
opts['rec_name'] = 'mSM65_09-Jul-2018' # Recording name
opts['frames_per_trial'] = 75 # Num of frames per trial
opts['dformat'] = 'MATLAB'
# Scoring parameters
opts['score_mets'] = ['Weighted R2', 'Old corr, float64', 'Old corr, float32', 'New corr'] # Scoring methods to test
opts['start'] = 9500 # Number of frames to use for each evaluation
opts['n_frames'] = 100 # Number of frames to use for each evaluation
opts['n_evals'] = 200 # Number of evaluations to perform
opts['alpha_met'] = 'fminbound' # The alpha evaluation method
opts['crossval_met'] = 'cross_val_predict' # The alpha evaluation method 
opts['sample_trials'] = -1 # Number of trials to randomally sample for R2 map calculation. -1 uses all of them.

### Load real and modeled data

In [33]:
data = rb.load_data(opts) # Load real widefield data
preds = rb.load_results('preds', opts)[0]  # Load modeled data prediction
pred = preds[opts['crossval_met'], opts['alpha_met']] # Loads the specific prediction corresponding to the chosen method
calc_score = rb.mint_calc_score(data) # Mint a custom score function weighted by component loadings

#### Benchmark scoring methods

In [42]:
scores = {}
score_times = {}
for score_method in tqdm(opts['score_mets'], desc='Alpha method num'):
    st = time.time()
    if score_method == 'Weighted R2':
        scores[score_method] = calc_score(data.svt, preds[opts['crossval_met'] , opts['alpha_met']])
    elif score_method == 'Old corr, float64':
        scores[score_method] = np.nanmean(rb.vis_score(data, preds[opts['crossval_met'] , opts['alpha_met']], opts, dtype=np.float64))
    elif score_method == 'Old corr, float32':
        scores[score_method] = np.nanmean(rb.vis_score(data, preds[opts['crossval_met'] , opts['alpha_met']], opts, dtype=np.float32))
    elif score_method == 'New corr':
        scores[score_method] = np.nanmean(rb.new_vis_score(data, preds[opts['crossval_met'] , opts['alpha_met']], opts))
    score_times[score_method] = time.time() - st

Alpha method num:   0%|          | 0/4 [00:00<?, ?it/s]

In [43]:
score_times

{'Weighted R2': 0.08078360557556152,
 'Old corr, float64': 0.8726661205291748,
 'Old corr, float32': 0.42984962463378906,
 'New corr': 0.3999300003051758}

In [44]:
scores

{'Weighted R2': 0.36875430291268874,
 'Old corr, float64': 0.3805236380070898,
 'Old corr, float32': 0.3805236,
 'New corr': 0.38052365}